In [55]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from textblob import TextBlob

In [57]:
class AdmissionChatbot:
    def __init__(self):
        self.context = {'program': None, 'concerns': []}
        self.programs = {
            'engineering': {
                'keywords': ['engineering', 'engineer'],
                'deadline': "September 15th",
                'requirements': "Academic transcripts, standardized test scores (SAT/ACT), letters of recommendation, and a personal statement focusing on your interest in engineering.",
                'courses': "Calculus, Physics, Chemistry, Introduction to Engineering",
                'faculty': "Dr. Jane Smith (Department Head), Dr. John Doe (Robotics Specialist)"
            },
            'business': {
                'keywords': ['business', 'commerce', 'management'],
                'deadline': "September 28th",
                'requirements': "Academic transcripts, standardized test scores (SAT/ACT), letters of recommendation, and a personal statement highlighting your leadership experiences.",
                'courses': "Economics, Accounting, Business Ethics, Marketing",
                'faculty': "Prof. Emily Johnson (Marketing), Dr. Michael Brown (Finance)"
            },
            'computer science': {
                'keywords': ['computer science', 'cs', 'informatics', 'computing'],
                'deadline': "September 1st",
                'requirements': "Academic transcripts with strong math and science grades, standardized test scores (SAT/ACT), letters of recommendation, and a personal statement describing your coding projects or experiences.",
                'courses': "Data Structures, Algorithms, Database Systems, Artificial Intelligence",
                'faculty': "Dr. Sarah Lee (AI Researcher), Prof. David Wang (Cybersecurity Expert)"
            },
            'medicine': {
                'keywords': ['medicine', 'medical', 'med','medicinal'],
                'deadline': "September 1st",
                'requirements': "Academic transcripts with pre-med coursework, MCAT scores, letters of recommendation, and a personal statement describing your motivation for pursuing medicine.",
                'courses': "Anatomy, Physiology, Biochemistry, Medical Ethics",
                'faculty': "Dr. Robert Chen (Neurology), Dr. Lisa Patel (Cardiology)"
            },
            'law': {
                'keywords': ['law', 'legal', 'jurisprudence'],
                'deadline': "September 20th",
                'requirements': "Academic transcripts, LSAT scores, letters of recommendation, and a personal statement outlining your interest in law and relevant experiences.",
                'courses': "Constitutional Law, Criminal Law, Civil Procedure, Legal Writing",
                'faculty': "Prof. Amanda Garcia (Criminal Law), Dr. Thomas Wilson (International Law)"
            },
            'psychology': {
                'keywords': ['psychology', 'psychological'],
                'deadline': "September 5th",
                'requirements': "Academic transcripts, standardized test scores (SAT/ACT), letters of recommendation, and a personal statement describing your interest in psychology and any research experience.",
                'courses': "Introduction to Psychology, Research Methods, Cognitive Psychology, Abnormal Psychology",
                'faculty': "Dr. Rachel Green (Clinical Psychology), Prof. Mark Taylor (Developmental Psychology)"
            },
            'arts': {
                'keywords': ['arts', 'fine arts', 'art'],
                'deadline': "September 10th",
                'requirements': "Academic transcripts, standardized test scores (SAT/ACT), letters of recommendation, a personal statement, and a portfolio of your work.",
                'courses': "Art History, Studio Art, Digital Media, Sculpture",
                'faculty': "Prof. Olivia White (Fine Arts), Dr. James Black (Art History)"
            }
        }
        nltk.download('punkt', quiet=True)
        nltk.download('stopwords', quiet=True)
        nltk.download('wordnet', quiet=True)
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()

    def start_conversation(self):
        print("Welcome to College Admission Chatbot! How can I assist you today?")
        while True:
            user_input = input("You: ").lower()
            if "goodbye" in user_input or "bye" in user_input:
                print("Chatbot: Thank you for using our admission chatbot. If you have any more questions in the future, feel free to ask. Goodbye!")
                break
            response = self.generate_response(user_input)
            print("Chatbot:", response)

    def preprocess_text(self, text):
        text = re.sub(r'[^\w\s]', '', text)
        tokens = word_tokenize(text)
        tokens = [self.lemmatizer.lemmatize(token) for token in tokens if token.isalnum()]
        tokens = [token for token in tokens if token not in self.stop_words]
        return tokens

    def generate_response(self, user_input):
        user_tokens = self.preprocess_text(user_input)
        sentiment = TextBlob(user_input).sentiment.polarity

        if "help" in user_tokens:
            return self.get_help_message()

        if sentiment < -0.2:
            return "I sense that you might be frustrated. I'm here to help. Could you please tell me more about your concerns regarding the admission process?"

        # Handle greetings and general inquiries
        greetings = ["hi", "hello", "hey", "greetings"]
        if any(word in user_tokens for word in greetings):
            return "Hello! Welcome to our college admissions chatbot. How can I assist you today? You can ask about admission requirements, procedures, deadlines, or specific programs."

        if "question" in user_tokens or "information" in user_tokens:
            return "I'd be happy to help! Our college offers admission to various programs including Engineering, Business, Computer Science, Medicine, Law, Psychology, and Arts. What specific information do you need? You can ask about requirements, procedures, deadlines, or particular programs."

        # Check context for personalization
        if self.context['program']:
            program = self.context['program']
            if "deadline" in user_tokens:
                return f"The deadline for the {program.capitalize()} program is {self.programs[program]['deadline']}. Do you need any other information about this program?"
            elif "requirement" in user_tokens:
                return f"The requirements for the {program.capitalize()} program are: {self.programs[program]['requirements']} Is there anything else you'd like to know?"
            elif "course" in user_tokens or "classes" in user_tokens:
                return f"Some key courses in the {program.capitalize()} program include: {self.programs[program]['courses']}. Would you like to know about the faculty?"
            elif "faculty" in user_tokens or "professor" in user_tokens:
                return f"Key faculty members in the {program.capitalize()} program include: {self.programs[program]['faculty']}. Is there anything else you'd like to know about this program?"

        # General Admission Information
        if any(word in user_tokens for word in ["requirement", "requirements","documents","required"]):
            return "General admission requirements typically include academic transcripts, standardized test scores (such as SAT or ACT), letters of recommendation, and a personal statement. The specific requirements may vary by program. Would you like to know about requirements for a particular program?"

        elif any(word in user_tokens for word in ["procedure", "process", "steps"]):
            return "The admission process typically involves: 1) Researching programs and requirements, 2) Taking necessary standardized tests, 3) Submitting an online application, 4) Paying the application fee, 5) Sending in required documents (transcripts, test scores, etc.), 6) Writing a personal statement, and 7) Waiting for the admission decision. "

        elif any(word in user_tokens for word in ["deadline", "deadlines"]):
            deadline_info = "Deadlines vary depending on the program. Here are some examples:\n"
            for program, details in self.programs.items():
                deadline_info += f"- {program.capitalize()}: {details['deadline']}\n"
            deadline_info += "Which program's deadline are you interested in?"
            return deadline_info

        elif any(word in user_tokens for word in ["fee", "cost", "payment"]):
            return "The application fee is typically Rs.50-Rs.100, but it may vary depending on the program and whether you're applying as a domestic or international student. Some applicants might be eligible for fee waivers based on financial need."

        # Program-Specific Information
        for program, details in self.programs.items():
            if any(keyword in user_tokens for keyword in details['keywords']):
                self.context['program'] = program  # Remember the program for context
                return f"The {program.capitalize()} program deadline is {self.programs[program]['deadline']}. The requirements include {self.programs[program]['requirements']} Would you like to know about courses, faculty, or any other specific information for this program?"

        # Farewell messages
        if any(word in user_tokens for word in ["thank", "thanks"]):
            return "You're welcome! I'm glad I could help. Is there anything else you'd like to know about our admissions process?"

        # Error Handling
        self.context['concerns'].append(user_input)
        return f"I'm sorry, I couldn't understand your question. Could you please rephrase it or ask about a specific aspect of college admissions, such as requirements, deadlines, or application procedures for a particular program? You can also type 'help' to see what kind of information I can provide."

    def get_help_message(self):
        return """
        Here are some ways I can help you:
        - Ask about admission requirements for specific programs
        - Inquire about application deadlines
        - Get information on the application process
        - Learn about application fees and waivers
        - Ask about specific programs like Engineering, Business, Computer Science, etc.
        - Get details about courses and faculty for each program
        Just ask your question, and I'll do my best to help!
        """


In [ ]:
if __name__ == "__main__":
    chatbot = AdmissionChatbot()
    chatbot.start_conversation()

Welcome to College Admission Chatbot! How can I assist you today?


You:  What is the admission procedure?


Chatbot: The admission process typically involves: 1) Researching programs and requirements, 2) Taking necessary standardized tests, 3) Submitting an online application, 4) Paying the application fee, 5) Sending in required documents (transcripts, test scores, etc.), 6) Writing a personal statement, and 7) Waiting for the admission decision. 


You:  What are the documents required?


Chatbot: General admission requirements typically include academic transcripts, standardized test scores (such as SAT or ACT), letters of recommendation, and a personal statement. The specific requirements may vary by program. Would you like to know about requirements for a particular program?


...

Chatbot: Key faculty members in the Medicine program include: Dr. Robert Chen (Neurology), Dr. Lisa Patel (Cardiology). Is there anything else you'd like to know about this program?


You:  What is the admission fee?


Chatbot: The application fee is typically Rs.50-Rs.100, but it may vary depending on the program and whether you're applying as a domestic or international student. Some applicants might be eligible for fee waivers based on financial need.


You:  Thank you


Chatbot: You're welcome! I'm glad I could help. Is there anything else you'd like to know about our admissions process?
